In [1]:
import numpy as np
import networkx as nx
import algorithms
import evaluate

Scheme works as follows:
1. Coarsen input graph with some __edge merging algorithm__
2. Partition coarsest graph with some __partitioning algorithm__
3. Project partition of coarsest graph to partition of input graph

# Test: SBM

In [2]:
sizes = np.array([100, 100, 100])
p = np.array([[.8, .1, .1],
              [.1, .8, .1],
              [.1, .1, .8]])
G = nx.stochastic_block_model(sizes=sizes, p=p)
A = nx.to_numpy_array(G)

In [3]:
def my_temperature_merging(A, node_weights):
    return algorithms.temperature_merging(A=A, node_weights=node_weights, temperature=.5)

In [4]:
merge_names = ['Temperature merging', 
               'Random matching', 
               'Heavy edge matching', 
               'Max-cut matching']

x_blocks = np.zeros(A.shape[0])
x_blocks[:sizes[0]] = 1
block_cut = evaluate.wcut(x=x_blocks, A=A)

x_rnd = np.random.binomial(n=1, p=.5, size=A.shape[0])
random_cut = evaluate.wcut(x=x_rnd, A=A)

uncoarsened_cut = evaluate.wcut(x=algorithms.compute_spectral_wcut(A), A=A)

print(f'Block cut: {block_cut}')
print(f'Random cut: {random_cut}')
print(f'Uncoarsened cut: {uncoarsened_cut}')  

for i, merge_fn in enumerate([my_temperature_merging,
                              algorithms.random_matching_merging,
                              algorithms.heavy_edge_merging, 
                              algorithms.max_cut_merging]):    
    print(f'Merging strategy: {merge_names[i]}')
    find_cuts = algorithms.FindCuts(A=A, merge_fn=merge_fn, partition_fn=algorithms.compute_spectral_wcut)
    for N_max in [100]:
        x = find_cuts(N_max=N_max)
        cut_val = evaluate.wcut(x=x, A=A)
        fractions = evaluate.evaluate_SBM_partition(x=x, sizes=sizes)
        print(f'N_max={N_max}: fractions {fractions} and cut value {cut_val}')

Block cut: 29.865
Random cut: 99.38650306748468
Uncoarsened cut: 29.52
Merging strategy: Temperature merging
Start coarsening
300 nodes in graph
169 nodes in graph
89 nodes in graph
Start partitioning
Start refining
N_max=100: fractions [0.95, 0.47, 0.24] and cut value 73.9480309296889
Merging strategy: Random matching
Start coarsening
300 nodes in graph
151 nodes in graph
76 nodes in graph
Start partitioning
Start refining
N_max=100: fractions [0.13, 0.18, 0.9] and cut value 63.23006602336211
Merging strategy: Heavy edge matching
Start coarsening
300 nodes in graph
151 nodes in graph
76 nodes in graph
Start partitioning
Start refining
N_max=100: fractions [0.82, 0.78, 0.19] and cut value 73.97848469458425
Merging strategy: Max-cut matching
Start coarsening
300 nodes in graph
151 nodes in graph
76 nodes in graph
Start partitioning
Start refining
N_max=100: fractions [0.41, 0.25, 0.91] and cut value 76.71373212774488


# Iterative algorithm to find coarser cuts

In [ ]:
class HierarchicalCuts(object):
    def __init__(self, A, CutFinder):
        self.A = A
        self.CutFinder = CutFinder